# AML 1

## Task 1

In [30]:
%pip install torchsummary
%pip install wandb
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00


In [7]:
!wandb login

wandb: Currently logged in as: js-agh (js-agh-agh). Use `wandb login --relogin` to force relogin


In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import wandb

In [9]:
wandb.login()

True

In [10]:
batch_size = 32

# Dataset import as in Ptytorch docs
ds_train = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
ds_test = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.3MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 212kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.94MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 20.8MB/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [11]:
# Defining the labels from datasetg - taken from kaggle
labels = {
    0: "T-shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [12]:
# Selecting the right device to perform the computations on
device = "cuda" if torch.cuda.is_available else "cpu"
print(f"Using {device} device")

Using cuda device


In [13]:
# Definition of the model


class FashionMnistClassifier(nn.Module):
    def __init__(self):
        super(FashionMnistClassifier, self).__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.drop1 = nn.Dropout2d(0.25)

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.drop2 = nn.Dropout2d(0.25)

        self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7 * 7 * 64, 512), nn.ReLU(), nn.Linear(512, 512), nn.ReLU(), nn.Linear(512, 10)
        )

    def forward(self, x):
        out = self.conv_block_1(x)
        out = self.drop1(out)
        out = self.conv_block_2(out)
        out = self.drop2(out)
        out = self.linear_relu_stack(self.flatten(out))
        return out

In [14]:
model = FashionMnistClassifier().to(device)
print(model.parameters())

<generator object Module.parameters at 0x7cde71ae8ba0>


In [15]:
# Settings
learning_rate = 1e-3
epochs = 5

loss = nn.CrossEntropyLoss()  # nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [16]:
def training_epoch(dataloader, model, loss, opt):
    model.train()  # Setting model into train mode
    samples = len(dataloader) * dataloader.batch_size
    processed = 0
    batch_num = 0

    for data, label in dataloader:
        data = data.to(device)
        label = label.to(device)

        opt.zero_grad()

        predictions = model(data)
        loss_result = loss(predictions, label)
        loss_result.backward()

        opt.step()

        processed += len(data)
        if batch_num % 100 == 0:
            cur_loss = loss_result.item()
            print(f"loss:{cur_loss:>7f} [{processed:>5d}/{samples:>5d}]")

        batch_num += 1

In [17]:
def evaluate(dataloader, model, loss):
    samples = len(dataloader.dataset)
    n_batches = len(dataloader)
    model.eval()  # Disenables storing training data

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, label in dataloader:
            data = data.to(device)
            label = label.to(device)

            predictions = model(data)

            loss_val = loss(predictions, label).item()
            correct += (predictions.argmax(1) == label).type(torch.float).sum().item()
            test_loss += loss_val
    test_loss /= n_batches
    accuracy = correct / samples

    return accuracy, test_loss, correct  # , cm


#     correct = 0
#     with torch.no_grad():
#         for data, label in dataloader:
#             data, label = data.to(device), label.to(device)
#             predictions = model(data)
#             correct += (predictions.argmax(1) == label).type(torch.float).sum().item()
#     accuracy = correct / len(dataloader.dataset)

In [18]:
def run_training(train, test, model, loss, opt, epochs):

    wandb.init(
      # Set the project where this run will be logged
      project="basic-intro",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment",
      # Track hyperparameters and run metadata
      config={
      # "learning_rate": 0.02,
      "architecture": "MNIST",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })

    for e in range(epochs):
        print(f"# Epoch {e}")
        training_epoch(train, model, loss, opt)
        train_accuracy, train_loss, train_correct = evaluate(train, model, loss)
        test_accuracy, test_loss, test_correct = evaluate(test, model, loss)
        print(
            f"train_loss = {train_loss}, train_acc = {train_accuracy}, test_loss = {test_loss}, test_acc = {test_accuracy}"
        )
        wandb.log({"acc": test_accuracy, "loss": test_loss})

In [19]:
from torchsummary import summary

summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
              ReLU-3           [-1, 32, 28, 28]               0
         MaxPool2d-4           [-1, 32, 14, 14]               0
         Dropout2d-5           [-1, 32, 14, 14]               0
            Conv2d-6           [-1, 64, 14, 14]          18,496
       BatchNorm2d-7           [-1, 64, 14, 14]             128
              ReLU-8           [-1, 64, 14, 14]               0
         MaxPool2d-9             [-1, 64, 7, 7]               0
        Dropout2d-10             [-1, 64, 7, 7]               0
          Flatten-11                 [-1, 3136]               0
           Linear-12                  [-1, 512]       1,606,144
             ReLU-13                  [-1, 512]               0
           Linear-14                  [

In [20]:
run_training(train_loader, test_loader, model, loss, optimizer, 20)

# Epoch 0
loss:2.281497 [   32/60000]
loss:2.282979 [ 3232/60000]
loss:2.163255 [ 6432/60000]
loss:2.105489 [ 9632/60000]
loss:1.963551 [12832/60000]
loss:1.809404 [16032/60000]
loss:1.593206 [19232/60000]
loss:1.432822 [22432/60000]
loss:1.377826 [25632/60000]
loss:1.264675 [28832/60000]
loss:1.115027 [32032/60000]
loss:1.059844 [35232/60000]
loss:1.051311 [38432/60000]
loss:1.152197 [41632/60000]
loss:1.136673 [44832/60000]
loss:0.900564 [48032/60000]
loss:0.755875 [51232/60000]
loss:0.979309 [54432/60000]
loss:0.868018 [57632/60000]
train_loss = 0.7297321080048879, train_acc = 0.7555666666666667, test_loss = 0.7351396874117013, test_acc = 0.7507
# Epoch 1
loss:0.812381 [   32/60000]
loss:0.939322 [ 3232/60000]
loss:0.771545 [ 6432/60000]
loss:0.709850 [ 9632/60000]
loss:0.857413 [12832/60000]
loss:0.921410 [16032/60000]
loss:0.515961 [19232/60000]
loss:0.767361 [22432/60000]
loss:0.723205 [25632/60000]
loss:0.544059 [28832/60000]
loss:0.692607 [32032/60000]
loss:0.557623 [35232/6000

## Task 2

In [21]:
class AutoEncoder(torch.nn.Module):
    def __init__(self, input_size):
        super(AutoEncoder, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16),
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(16, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, input_size),
            torch.nn.ReLU(),
        )

    def forward(self, x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out

In [26]:
ds_train = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
ds_test = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())
batch_size = 32
train_dataloader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(ds_test, batch_size=batch_size, shuffle=True)

model = AutoEncoder(28 * 28)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-1, weight_decay=1e-8)

loss_f = torch.nn.MSELoss()

# Selecting the right device to perform the computations on
device = "cuda" if torch.cuda.is_available else "cpu"
print(f"Using {device} device")

model = model.to(device)

Using cuda device


In [23]:
def one_epoch_train(model, optimizer, loss, train_dataloader, device):
    model.train()
    processed_images = 0
    current_batch_index = 0
    samples = len(train_dataloader.dataset)
    for image, _ in train_dataloader:
        image = image.to(device)
        image = image.reshape(-1, 28 * 28)

        reconstructed_image = model(image)
        computed_loss = loss(reconstructed_image, image)
        computed_loss.backward()

        optimizer.step()

        processed_images += len(image)
        if current_batch_index % 100 == 0:
            current_loss_value = computed_loss.item()
            print(f"loss:{current_loss_value:>7f} [{processed_images:>5d}/{samples:>5d}]")
        current_batch_index += 1

In [24]:
def evaluate(model, loss, dataloader, device):
    model.eval()
    samples = len(dataloader.dataset)
    n_batches = len(dataloader)
    total_loss = 0
    samples = len(dataloader.dataset)
    for image, _ in dataloader:
        image = image.to(device)
        image = image.reshape(-1, 28 * 28)

        reconstructed_image = model(image)
        computed_loss = loss(reconstructed_image, image)
        total_loss += computed_loss.item()
    return total_loss / n_batches

In [28]:
def run_training(train, test, model, loss, opt, epochs):
    wandb.init(
      # Set the project where this run will be logged
      project="task2-autoencoder",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment",
      # Track hyperparameters and run metadata
      config={
      # "learning_rate": 0.02,
      "architecture": "Autoencoder",
      "dataset": "FashionMnist",
      "epochs": 10,
      })
    for e in range(epochs):
        print(f"# Epoch {e}")
        one_epoch_train(model, opt, loss, train, device)
        train_loss = evaluate(model, loss, train, device)
        test_loss = evaluate(model, loss, test, device)
        print(f"train_loss = {train_loss}, test_loss = {test_loss}")
        wandb.log({"loss": test_loss})


run_training(train_dataloader, test_dataloader, model, loss_f, optimizer, 20)

# Epoch 0
loss:0.216613 [   32/60000]
loss:0.231600 [ 3232/60000]
loss:0.217344 [ 6432/60000]
loss:0.209716 [ 9632/60000]
loss:0.196034 [12832/60000]
loss:0.206638 [16032/60000]
loss:0.181775 [19232/60000]
loss:0.196764 [22432/60000]
loss:0.227923 [25632/60000]
loss:0.209154 [28832/60000]
loss:0.207005 [32032/60000]
loss:0.210441 [35232/60000]
loss:0.201638 [38432/60000]
loss:0.212891 [41632/60000]
loss:0.211371 [44832/60000]
loss:0.202502 [48032/60000]
loss:0.195790 [51232/60000]
loss:0.211923 [54432/60000]
loss:0.219100 [57632/60000]
train_loss = 0.20644534362951913, test_loss = 0.20640322808830883
# Epoch 1
loss:0.171770 [   32/60000]
loss:0.220140 [ 3232/60000]
loss:0.219624 [ 6432/60000]
loss:0.200138 [ 9632/60000]
loss:0.211830 [12832/60000]
loss:0.192850 [16032/60000]
loss:0.182935 [19232/60000]
loss:0.196765 [22432/60000]
loss:0.169959 [25632/60000]
loss:0.190635 [28832/60000]
loss:0.210318 [32032/60000]
loss:0.221196 [35232/60000]
loss:0.207850 [38432/60000]
loss:0.194765 [416

In [31]:
def run_training_with_optuna(train, test, model_class, loss_fn, epochs, device, n_trials=50):
    """
    Runs training with Optuna hyperparameter optimization.

    Parameters:
    - train: Training DataLoader
    - test: Testing DataLoader
    - model_class: Model class (e.g., AutoEncoder)
    - loss_fn: Loss function
    - epochs: Number of epochs for each trial
    - device: Device to run the training on ('cpu' or 'cuda')
    - n_trials: Number of Optuna trials

    Returns:
    - best_model: The model with the best performance
    - best_params: Dictionary of the best hyperparameters
    """
    import optuna

    def objective(trial):
        # Suggest hyperparameters
        learning_rate = trial.suggest_loguniform("lr", 1e-4, 1e-1)
        weight_decay = trial.suggest_loguniform("weight_decay", 1e-8, 1e-4)

        # Initialize model and optimizer
        model = model_class().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        # Training loop for the trial
        for e in range(epochs):
            one_epoch_train(model, optimizer, loss_fn, train, device)

        # Evaluate on test data
        test_loss = evaluate(model, loss_fn, test, device)

        # Log test loss for the trial
        wandb.log({"trial_test_loss": test_loss, "trial": trial.number})
        return test_loss  # Optuna minimizes this loss

    # Start the Optuna study
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)

    # Log Optuna results
    print("Best trial:", study.best_trial.number)
    print("Best hyperparameters:", study.best_params)

    # Train the final model with the best parameters
    best_params = study.best_params
    best_model = model_class().to(device)
    best_optimizer = torch.optim.Adam(
        best_model.parameters(),
        lr=best_params["lr"],
        weight_decay=best_params["weight_decay"],
    )

    # Run the final training
    wandb.init(
        project="task2-autoencoder-optuna",
        name="final_model_training",
        config={
            "learning_rate": best_params["lr"],
            "weight_decay": best_params["weight_decay"],
            "architecture": "Autoencoder",
            "dataset": "FashionMNIST",
            "epochs": epochs,
        },
    )
    for e in range(epochs):
        print(f"# Final Epoch {e}")
        one_epoch_train(best_model, best_optimizer, loss_fn, train, device)
        train_loss = evaluate(best_model, loss_fn, train, device)
        test_loss = evaluate(best_model, loss_fn, test, device)
        print(f"Final train_loss = {train_loss}, Final test_loss = {test_loss}")
        wandb.log({"final_train_loss": train_loss, "final_test_loss": test_loss})

    return best_model, study.best_params

device = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn = torch.nn.MSELoss()

best_model, best_hyperparameters = run_training_with_optuna(
    train_dataloader,
    test_dataloader,
    lambda: AutoEncoder(28 * 28),
    loss_fn,
    epochs=20,
    device=device,
    n_trials=50
)

print("Training complete!")
print("Best Hyperparameters:", best_hyperparameters)


[I 2024-11-19 01:00:08,832] A new study created in memory with name: no-name-ee391ced-3556-4b1f-9c01-06db67ae4526
<ipython-input-31-b06a9f51926b>:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("lr", 1e-4, 1e-1)
<ipython-input-31-b06a9f51926b>:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-8, 1e-4)


loss:0.182395 [   32/60000]
loss:0.126343 [ 3232/60000]
loss:0.134908 [ 6432/60000]
loss:0.148790 [ 9632/60000]
loss:0.139945 [12832/60000]
loss:0.126435 [16032/60000]
loss:0.119933 [19232/60000]
loss:0.113512 [22432/60000]
loss:0.093908 [25632/60000]
loss:0.077546 [28832/60000]
loss:0.085704 [32032/60000]
loss:0.099657 [35232/60000]
loss:0.113802 [38432/60000]
loss:0.117418 [41632/60000]
loss:0.119010 [44832/60000]
loss:0.108415 [48032/60000]
loss:0.095534 [51232/60000]
loss:0.098938 [54432/60000]
loss:0.092423 [57632/60000]
loss:0.107880 [   32/60000]
loss:0.069950 [ 3232/60000]
loss:0.089356 [ 6432/60000]
loss:0.086499 [ 9632/60000]
loss:0.102219 [12832/60000]
loss:0.096251 [16032/60000]
loss:0.088142 [19232/60000]
loss:0.082744 [22432/60000]
loss:0.088250 [25632/60000]
loss:0.097942 [28832/60000]
loss:0.093467 [32032/60000]
loss:0.085107 [35232/60000]
loss:0.082323 [38432/60000]
loss:0.090990 [41632/60000]
loss:0.087592 [44832/60000]
loss:0.102979 [48032/60000]
loss:0.077205 [51232

[I 2024-11-19 01:04:19,611] Trial 0 finished with value: 0.09418660452285894 and parameters: {'lr': 0.00012623750222694592, 'weight_decay': 2.0073199367240873e-08}. Best is trial 0 with value: 0.09418660452285894.


loss:0.223430 [   32/60000]
loss:0.096622 [ 3232/60000]
loss:0.109061 [ 6432/60000]
loss:0.102604 [ 9632/60000]
loss:0.118645 [12832/60000]
loss:0.115257 [16032/60000]
loss:0.127240 [19232/60000]
loss:0.131146 [22432/60000]
loss:0.151076 [25632/60000]
loss:0.146063 [28832/60000]
loss:0.141904 [32032/60000]
loss:0.154731 [35232/60000]
loss:0.170949 [38432/60000]
loss:0.160724 [41632/60000]
loss:0.157014 [44832/60000]
loss:0.149599 [48032/60000]
loss:0.167875 [51232/60000]
loss:0.182501 [54432/60000]
loss:0.203986 [57632/60000]
loss:0.170982 [   32/60000]
loss:0.201785 [ 3232/60000]
loss:0.204253 [ 6432/60000]
loss:0.177552 [ 9632/60000]
loss:0.160505 [12832/60000]
loss:0.204224 [16032/60000]
loss:0.165168 [19232/60000]
loss:0.231081 [22432/60000]
loss:0.195175 [25632/60000]
loss:0.204228 [28832/60000]
loss:0.226128 [32032/60000]
loss:0.213939 [35232/60000]
loss:0.212160 [38432/60000]
loss:0.194513 [41632/60000]
loss:0.222864 [44832/60000]
loss:0.230855 [48032/60000]
loss:0.206970 [51232

[I 2024-11-19 01:08:20,284] Trial 1 finished with value: 0.20647229087619354 and parameters: {'lr': 0.0031648310418717976, 'weight_decay': 1.1575871918433289e-05}. Best is trial 0 with value: 0.09418660452285894.


...

loss:0.271554 [   32/60000]
loss:0.145074 [ 3232/60000]
loss:0.130078 [ 6432/60000]
loss:0.109557 [ 9632/60000]
loss:0.125961 [12832/60000]
loss:0.113029 [16032/60000]
loss:0.117976 [19232/60000]
loss:0.095297 [22432/60000]
loss:0.108175 [25632/60000]
loss:0.099787 [28832/60000]
loss:0.099154 [32032/60000]
loss:0.100580 [35232/60000]
loss:0.099194 [38432/60000]
loss:0.091872 [41632/60000]
loss:0.091855 [44832/60000]
loss:0.111253 [48032/60000]
loss:0.109843 [51232/60000]
loss:0.102533 [54432/60000]
loss:0.092832 [57632/60000]
loss:0.100616 [   32/60000]
loss:0.103856 [ 3232/60000]
loss:0.101180 [ 6432/60000]
loss:0.099523 [ 9632/60000]
loss:0.106404 [12832/60000]
loss:0.099532 [16032/60000]
loss:0.094879 [19232/60000]
loss:0.092572 [22432/60000]
loss:0.098252 [25632/60000]
loss:0.098707 [28832/60000]
loss:0.102589 [32032/60000]
loss:0.102115 [35232/60000]
loss:0.095735 [38432/60000]
loss:0.106490 [41632/60000]
loss:0.109143 [44832/60000]
loss:0.100902 [48032/60000]
loss:0.105605 [51232

[I 2024-11-19 02:32:13,992] Trial 22 finished with value: 0.10240216203295766 and parameters: {'lr': 0.0003984206618861179, 'weight_decay': 7.17488035798507e-07}. Best is trial 0 with value: 0.09418660452285894.


loss:0.198038 [   32/60000]
loss:0.124382 [ 3232/60000]
loss:0.123644 [ 6432/60000]
loss:0.123323 [ 9632/60000]
loss:0.113782 [12832/60000]
loss:0.095238 [16032/60000]
loss:0.089593 [19232/60000]
loss:0.095230 [22432/60000]
loss:0.101263 [25632/60000]
loss:0.095391 [28832/60000]
loss:0.092975 [32032/60000]
loss:0.092948 [35232/60000]
loss:0.096165 [38432/60000]
loss:0.101087 [41632/60000]
loss:0.099422 [44832/60000]
loss:0.096423 [48032/60000]
loss:0.088719 [51232/60000]
loss:0.111298 [54432/60000]
loss:0.096651 [57632/60000]
loss:0.099021 [   32/60000]
loss:0.091283 [ 3232/60000]
loss:0.090025 [ 6432/60000]
loss:0.101975 [ 9632/60000]
loss:0.095875 [12832/60000]
loss:0.108691 [16032/60000]
loss:0.090600 [19232/60000]
loss:0.092813 [22432/60000]
loss:0.112570 [25632/60000]
loss:0.102321 [28832/60000]
loss:0.100399 [32032/60000]
loss:0.101726 [35232/60000]
loss:0.097052 [38432/60000]
loss:0.119502 [41632/60000]
loss:0.103080 [44832/60000]
loss:0.092562 [48032/60000]
loss:0.092622 [51232

[W 2024-11-19 02:34:34,626] Trial 23 failed with parameters: {'lr': 0.0008302411727390355, 'weight_decay': 2.114836973944019e-06} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-31-b06a9f51926b>", line 31, in objective
    one_epoch_train(model, optimizer, loss_fn, train, device)
  File "<ipython-input-23-33c9e6924adb>", line 10, in one_epoch_train
    reconstructed_image = model(image)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "<ipython-input-21-95439b391bc9>", line 27, in forward
    out = self.decoder(out)
  File "/usr/local/lib/pyt

KeyboardInterrupt: 